✨This a Jupyter Notebook that allows you to run Python code interactively 🐍✨

## Getting Started
1. Click the **Select Kernel** button on the top right of the notebook.
2. Choose **Python environments** and select **Python 3.11.9**.

Run code in the notebook by clicking the play button on the left side of the code cells.

## Learning Outcomes
We will focus on four key outcomes, each split into their own notebook:

1. [Understanding agents and prompt engineering with Prompty.](#1-understanding-agents-and-prompt-engineering-with-prompty)
2. [Utilizing Prompty tracing for debugging and observability.](#2-utilizing-prompty-tracing-for-debugging-and-observabilty)
3. [Building and running Contoso Creative Writer.](#3-building-and-running-contoso-creative-writer)
4. [Setting up automated evaluations with GitHub Actions.](#4-setting-up-automated-evaluations-and-deployment-with-github-actions)

Let’s get started!

## 1. Understanding Agents and Prompt Engineering with Prompty
### 1.1. What are AI agents?
Contoso Creative Writer is an Agentic Application. 

**In artificial intelligence an agent is a program designed to:**

- Perceive its environment
- Make decisions
- Take actions to achieve specific goals

For Contoso Creative Writer, the goal is to help the marketing team at Contoso Outdoors write well-researched, product-specific articles. 
<br>Contoso Creative Writer is made up of 4 agents that help achieve this goal. 

<img src="../../images/agents.png" alt="Agents in Contoso Creative Writer" width="900" height="380">

### 1.2. How is an AI agent built?

Each agent in Contoso Creative Writer is built with [Prompty](https://prompty.ai/)! 

#### 1.2.1 What is Prompty?
Prompty is a new asset class and file format for LLM prompts that aims to provide observability, understandability, and portability for developers.

**The Prompty file:**
- A Prompty file is not tied to any language as it uses the markdown format with YAML
- The file contains two main parts:
    - **Front Mattter:** 
        - This is the first part of the prompty file 
        - It is written in YAML and is contained inside two `---` seperators. 
        - It includes basic details about the prompt, the model configuration and prompty inputs. 

    - **Prompt Template:** 
        - This is the base prompt that is sent to the LLM once the prompty is executed. 
        - It uses Jinja format to pass values either specified in the front matter or from the application to the LLM.
        - Given *'name': Marlene*, the variable *{{name}}* will be replaced by *Marlene* at runtime. 

**The VS Code extension tool:**
- The Prompty extension allows you to run Prompty files directly in VS Code. 
- It has been pre-installed for this workshop, but you can also find it in the Visual Studio Code Marketplace.


### 1.3. Building an AI Agent

To help us understand practically how we build an AI agent will build the **Researcher Agent** step by step.
<br>In order to build the Researcher agent you will complete the following 3 steps.


#### **Step 1:** Build a multi-lingual query generator

- The researcher agent generates queries that can be used to look for information online. 
- It also allows us to find search results in multiple languages. 

---
<img src="todo.png" alt="todo icon" width="40" height="40"> **Tasks for you to do:**

> **TODO 1:** Open the [researcher-0.prompty](researcher/researcher-0.prompty) file, read the prompt and **click the play button** on the top right of the file.

At the top of the file you should see the following instructions in the sample section:
<br>***instructions:** Can you generate queries to find the latest camping trends and what folks are doing in the winter? Use 'en-US' as the market code.*
 
 > **TODO 2:** Edit the instructions to use a new language. (For example use *es-ES* instead of *en-US*, to get the results back in Spanish)

<img src="coding.png" alt="todo icon" width="45" height="45"> **Run the code:**

You can also execute a Prompty file using the Prompty Python package. 

> **TODO 3:** Click the play button to the left of the cell to run *researcher-0.prompty* file in Python.  

In [ ]:
import prompty
import os
from IPython.display import Markdown

instructions = "Can you generate queries to find the latest camping trends and what folks are doing in the winter? Use 'en-US' as the market code. "
Markdown(prompty.execute(os.getcwd() + "/researcher/researcher-0.prompty", inputs={"instructions": instructions}))

---

##### **Step 2:** Give the query generator a list of functions 
- In order for the researcher to generate even better queries it needs to know which search functions are avaialble to it. 
- Prompty allows us to pass information to the LLM in the form of a json file using the *${file:filename.json}* format.
- In this step we want to give the LLM a list of the functions (sometimes called tools) that it can choose from.
- Open the [functions.json](researcher/functions.json) file. Read the descriptions of the **find_information**, **find_entities** and **find_news** functions. 
- Open the [researcher-1.prompty](researcher/researcher-1.prompty) file and note that *${file:functions.json}* has been added to *tools* under the *parameters* section in the front matter. 
- Click the run button on the top right of the file and look at the results in the output tab.

**Passing a json file to the *tools* parameter in Prompty invokes the LLM to:**
- return a list of dictionaries, where each dictionary represents a function to call and its arguments
- the *funcion* key contains the generated query and the selected market as arguments
- the *name* key contains the name of the most appropriate function to use

✅ To complete this step update the instructions passed in the *sample* section of the prompty file to influence which function the LLM chooses. 
<br>(For example to influence choice of the *find_news* function, use instructions: 'Can you find the latest news about Microsoft?')

##### **Step 3:** Understanding how the LLM chooses which function to call

**Let's walk through the code**

Let's import `execute_researcher_prompty` from the [researcher3.py](./researcher/researcher3.py) script:

In [16]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/researcher'))

from researcher3 import execute_researcher_prompty

The Prompty template is configured to load a catalog [functions.json](./researcher/functions.json) with the 3 functions:
- `find_information`
- `find_entities`
- `find_news`

Let's test each of one them to get a sense of how the LLM selects them

##### a.1. Find information

Let's see how the LLM can help us find information on the web. It cannot find information per se but it can select amongst a catalog of functions the one which would allow achieving that goal.

In [ ]:
instructions = "Can you find the best educational material for learning Python programming"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

The `find_information` function was selected by the LLM based on the description of the function. It also figured out which parameter values should be passed to the function.

Go have a look at the definition of the `find_information` function in [functions.json](./researcher/functions.json) and try to get a sense of why this function was selected.

##### a.2. Find entities

In [ ]:
instructions = "Who is the inventor of the Python programming language?"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

Is the `find_entities` function called? Why? Go look at the `find_entities` definition in [functions.json](./researcher/functions.json).

##### a.3. Find news

In [ ]:
instructions = "What's the latest about the Python programming language?"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

🐞**BUG ALERT:** A bug has purposefully been left in the [functions.json](./researcher/functions.json) file.

Which function call has been selected? If any? With which parameters? Is it the `find_information` function we want?

**Exercice**:
- Find why the `find_news` function is not the one selected by the LLM
- Learn how to steer the LLM to select a given function using its description and parameters
- Add the function definition for `find_news` to the [functions.json](./researcher/functions.json) file
<details>
  <summary>find_news function definition</summary>
  
```json

  {
    "type": "function",
    "function": {
      "name": "find_news",
      "description": "Finds jokes on the web given a query. This function uses the Bing Search API to find jokes on the web given a query. The response includes the funniest jokes from the web and should be used if you're looking for a laugh.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "An optimal search query to find jokes on the web using the Bing Search API"
          },
          "market": {
            "type": "string",
            "description": "The market to search in, e.g. en-US - it should match the language of the query"
          }
        },
        "required": [
          "query"
        ]
      }
    }
  }
```
</details>

##### **Step 4:** Build the functions and execute the research
- The researcher has now selected which function to use and has generated a query and market code to pass to it. 
- We now need to write the Python code for these functions that will pass the queries to the Bing Search API. 
- We will also use a function in Python to execute the prompty file, instead of running it manually in VS Code. 
- To put everything together click the play button on the left of the jupyter notebook code cell below below.  
- The notebook cell calls the *research* function to run code from **researcher3.py**.
- Open the the [researcher3.py](researcher/researcher3.py) file to see this code and try and understand what each function does. 

✅ To complete this step run the code in the cell below and test it out with different instructions!

In [ ]:
import sys
import os
from IPython.display import JSON

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/researcher'))

from researcher3 import execute_function_calls

instructions = "Can you find the best educational material for learning Python programming"

# Execute the researcher prompty to get a list of functions calls
function_calls = execute_researcher_prompty(instructions=instructions)

# Execute the function calls
research = execute_function_calls(function_calls)
research

##### Congratulations you've build your first AI Agent with Prompty🎉
- [✅] Step 1: Build a multi-lingual query generator
- [✅] Step 2: Give the query generator a list of functions available
- [✅] Step 3: Build the tools and execute the research

We can now succesfully move on to learning outcome 2. 

➡️ **Open [next workshop notebook](./workshop-2-tracing.ipynb)**